[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HSinger04/VOGUE-Reimplementation/blob/main/cryu854/Image2Latent.ipynb)

In [1]:
%cd /content
!git clone https://github.com/HSinger04/VOGUE-Reimplementation

/content
Cloning into 'VOGUE-Reimplementation'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 450 (delta 96), reused 0 (delta 0), pack-reused 303
Receiving objects: 100% (450/450), 83.92 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (262/262), done.


In [2]:
%cd /content/VOGUE-Reimplementation/cryu854/

/content/VOGUE-Reimplementation/cryu854


## Mount drive for dataset and weights

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Imports

In [12]:
import tensorflow as tf
import os
from tensorflow.keras.layers import \
BatchNormalization, ELU, Conv2D, Input

## Load data

In [5]:
# Pro: Save space in google drive
# Contra: Needs to load the whole dataset every time
# use list_files to get generate dataset from data in different subdirectories easily
data = tf.data.Dataset.list_files("/content/drive/MyDrive/Lernen/Coxi/IANNwTF/ffhq-dataset/images1024x1024/" 
                                  + "[0-9]" * 2
                                  + "000/*.png")

## Convert paths to images

In [6]:
def decode_ffhq(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    return img

# TODO: Convert to appropriate dtype
data = data.map(decode_ffhq)

# TODO: Some better image stuff

# Loading trained model

## Actually load the model

In [7]:
!git pull
import tensorflow as tf
from modules.generator import generator

resolution = 1024  
config = "f"
num_labels = 0
checkpoint_path = "/content/drive/MyDrive/Lernen/Coxi/IANNwTF/official_1024x1024/"

Gs = generator(resolution, num_labels, config, randomize_noise=False)
ckpt = tf.train.Checkpoint(generator_clone=Gs)
print(f'Loading network from {checkpoint_path}...')
ckpt.restore(tf.train.latest_checkpoint(checkpoint_path)).expect_partial()

Already up to date.
Loading network from /content/drive/MyDrive/Lernen/Coxi/IANNwTF/official_1024x1024/...


## Train model

Layers & models also feature a boolean attribute trainable. Its value can be changed. Setting layer.trainable to False moves all the layer's weights from trainable to non-trainable.

In [8]:
truncation_psi = 0.5
batch_size = 5
latent_size = 512
latents = tf.random.normal([batch_size, latent_size])
# TODO: from _get_labels
labels_indice = [0]*batch_size
labels = tf.zeros([batch_size, 0], tf.float32)
images = Gs([latents, labels], truncation_psi=truncation_psi, training=False)

In [9]:
print(images.dtype)
print(images[0])
print(tf.math.reduce_min(images[0]))
print(tf.math.reduce_max(images[0]))

<dtype: 'float32'>
tf.Tensor(
[[[-0.46307448 -0.5302782  -0.6133043 ]
  [-0.60044396 -0.7186482  -0.8310894 ]
  [-0.6716418  -0.7745461  -0.8692887 ]
  ...
  [-0.7033296  -0.7787447  -0.82294476]
  [-0.69020903 -0.7829868  -0.84107673]
  [-0.6095885  -0.68863875 -0.75412345]]

 [[-0.6323254  -0.65564185 -0.6911636 ]
  [-0.7574098  -0.81197846 -0.84890676]
  [-0.7640804  -0.82685584 -0.8691092 ]
  ...
  [-0.7139143  -0.78837645 -0.82950306]
  [-0.69122887 -0.7851502  -0.8345704 ]
  [-0.6798398  -0.7710924  -0.8295144 ]]

 [[-0.7315802  -0.7622422  -0.7975961 ]
  [-0.7815012  -0.82403034 -0.8604169 ]
  [-0.74959934 -0.80666304 -0.84258854]
  ...
  [-0.730462   -0.792487   -0.85261977]
  [-0.74127316 -0.8133509  -0.8715573 ]
  [-0.75548244 -0.8055092  -0.858563  ]]

 ...

 [[ 0.21244687  0.12159576  0.0272996 ]
  [ 0.19183806  0.13988495  0.03687358]
  [ 0.21783781  0.1702074   0.07957324]
  ...
  [ 0.05248946  0.02969342 -0.03884888]
  [ 0.11060014  0.08011213 -0.03334475]
  [ 0.07815439

In [10]:
import sklearn
from matplotlib import pyplot as plt

# TODO: proper image displaying
for i in range(batch_size):
    temp = images[i]
    maxim = images.max
    minim = images.min
    temp = 
    plt.imshow(sklearn.preprocessing.normalize(images[i]))

AttributeError: ignored

In [15]:
encoder = tf.keras.Sequential(
    [
     Input((1024, 1024, 3)),
     Conv2D(2, 2, strides=2),
     Conv2D(2, 2, strides=2),
     Conv2D(2, 2, strides=2)
    ]
)

encoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 512, 512, 2)       26        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 2)       18        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 2)       18        
Total params: 62
Trainable params: 62
Non-trainable params: 0
_________________________________________________________________
